In [21]:
import csv
import os
import numpy as np

In [2]:
def create_mimic_pair_file(input_dir,species_name,deleted_prefix,file_names):
    for file in os.scandir(input_dir):
        #print(file.path.removeprefix(input_dir+"/"))
        
        if file.is_dir() and file.path.removeprefix(input_dir+"/")==species_name:
            for f in os.scandir(file):
                if f.is_file():
                    file_names.append(f.path.removeprefix(deleted_prefix)) 
        elif file.is_dir():
            create_mimic_pair_file(file.path,species_name,deleted_prefix,file_names)
    return file_names

In [104]:
notebook_path=os.path.dirname(os.path.realpath("Experiment4.ipynb"))
input_dir=notebook_path+"/ButterFly/"
deleted_prefix=input_dir
pair_folder_location=notebook_path+"/Experiment4/Pairs_text/"

        
with open ('mimic_pairs.csv') as csv_file:
    csv_reader=csv.reader(csv_file,delimiter=",")
    for row in csv_reader:
        print(row[0],row[1])
        mimic_pair_1_image_names=[]
        mimic_pair_2_image_names=[]
        
        mimic_pair_1_image_names=create_mimic_pair_file(input_dir,row[0],deleted_prefix,mimic_pair_1_image_names)
        mimic_pair_2_image_names=create_mimic_pair_file(input_dir,row[1],deleted_prefix,mimic_pair_2_image_names)
        
        
        self1_pair_str_name=pair_folder_location+row[0]+"_self.txt"
        count=0
        f=open(self1_pair_str_name, 'w')
        for i in range(len(mimic_pair_1_image_names)):
            for j in range(i+1,len(mimic_pair_1_image_names)):
                count+=1
                f.write(mimic_pair_1_image_names[i] + " "+mimic_pair_1_image_names[j]+"\n")
        print(count)
        f.close()
        
        self2_pair_str_name=pair_folder_location+row[1]+"_self.txt"
        f=open(self2_pair_str_name, 'w')
        for i in range(len(mimic_pair_2_image_names)):
            for j in range(i+1,len(mimic_pair_2_image_names)):
                f.write(mimic_pair_2_image_names[i] + " "+mimic_pair_2_image_names[j]+"\n")
        f.close()
        
        cross_pair_str_name=pair_folder_location+row[0]+"_"+row[1]+"_cross.txt"
        f=open(cross_pair_str_name, 'w')
        for i in range(len(mimic_pair_1_image_names)):
            for j in range(len(mimic_pair_2_image_names)):
                f.write(mimic_pair_1_image_names[i] + " "+mimic_pair_2_image_names[j]+"\n")
        f.close()

erato_petiverana melpomene_rosina_N
903
erato_petiverana melpomene_rosina_S
903
erato_hydara melpomene_melpomene
1225
erato_venus melpomene_vulcanus
45
erato_cyrbia melpomene_cythera
231
erato_notabilis melpomene_plesseni
21
erato_etylus melpomene_ecuadorensis
21
erato_emma melpomene_aglaope
1
erato_favorinus melpomene_amaryllis
6
erato_microclea melpomene_xenoclea
45
erato_luscombei melpomene_schunkei
3
erato_phyllis melpomene_burchelli
1770
erato_phyllis melpomene_nanna
1770


In [40]:
import re

notebook_path=os.path.dirname(os.path.realpath("Experiment4.ipynb"))
npz_folder_location=notebook_path+"/Experiment4/SuperGlue_output_files/SuperGlue_output_files/"
heat_map_path=notebook_path+"/Experiment4/heat_map/"

for folder in os.scandir(npz_folder_location):
    if folder.is_dir():
        print(folder.name)
        save_heat_map_path_for_current_folder=heat_map_path+folder.name+"/"
        if not os.path.exists(save_heat_map_path_for_current_folder):
            os.makedirs(save_heat_map_path_for_current_folder)
            
        set_of_image_names=[]
        
        for file in os.scandir(folder.path):
            print(file.name[0:int([m.start() for m in re.finditer(r"_",file.name)][2])])
            
            name_of_image=file.name[0:int([m.start() for m in re.finditer(r"_",file.name)][2])]
            
            if set_of_image_names.count(name_of_image)==0:
                set_of_image_names.append(name_of_image)
                heatmap=np.zeros((1600,1600))
                c=0
                
                for file2 in os.scandir(folder.path):
                    name_without_ext=file2.name.rsplit( ".", 1 )[ 0 ]
                    start_ind=int([m.start() for m in re.finditer(r"_",name_without_ext)][2])+1
                    end_ind=int([m.start() for m in re.finditer(r"_",name_without_ext)][5])
                    second_file_name=name_without_ext[start_ind:end_ind]
                    
                    #if the first file is the one we are doing heatmap for
                    if file2.name.startswith(name_of_image):
                        c=c+1
                        npz=np.load(folder.path+"/"+file2.name)
                        for i in range(npz['keypoints0'].shape[0]):
                            if npz['matches'][i]>-1:
                                heatmap[int(npz['keypoints0'][i][1])][int(npz['keypoints0'][i][0])]+=1
                                
                    #if the second file is the one we are doing heatmap for
                    elif name_of_image == second_file_name:
                        c=c+1
                        npz=np.load(folder.path+"/"+file2.name)
                        for i in range(npz['keypoints0'].shape[0]):
                            if npz['matches'][i]>-1:
                                heatmap[int(npz['keypoints1'][npz['matches'][i]][1])][int(npz['keypoints1'][npz['matches'][i]][0])]+=1
                    
                np.savez(save_heat_map_path_for_current_folder+name_of_image, heat_map=heatmap)
                        
                print(c)   
            
        


erato_notabilis_self
10429024_D_lowres
6
10429021_D_lowres
6
10429016_D_lowres
6
10429012_D_lowres
6
10429021_D_lowres
10429024_D_lowres
10429015_D_lowres
6
10429026_D_lowres
6
10429015_D_lowres
10429024_D_lowres
10429016_D_lowres
10429015_D_lowres
10429024_D_lowres
10429021_D_lowres
10429016_D_lowres
10429024_D_lowres
10429016_D_lowres
10429015_D_lowres
10429015_D_lowres
10429015_D_lowres
10429012_D_lowres
melpomene_vulcanus_self
10428019_D_lowres
10
10428017_D_lowres
10
10428025_D_lowres
10
10428017_D_lowres
10428017_D_lowres
10428019_D_lowres
10428029_D_lowres
10
10428025_D_lowres
10428019_D_lowres
10428020_D_lowres
10
10428017_D_lowres
10428022_D_lowres
10
10428022_D_lowres
10428022_D_lowres
10428025_D_lowres
10428025_D_lowres
10428024_D_lowres
10
10428021_D_lowres
10
10428021_D_lowres
10428017_D_lowres
10428024_D_lowres
10428020_D_lowres
10428024_D_lowres
10428020_D_lowres
10428022_D_lowres
10428024_D_lowres
10428029_D_lowres
10428032_D_lowres
10
10428020_D_lowres
10428022_D_lowre

33
10428158_D_lowres
10428139_D_lowres
33
10428157_D_lowres
33
10428160_D_lowres
33
10428142_D_lowres
33
10428148_D_lowres
33
10428151_D_lowres
10428168_D_lowres
33
10428148_D_lowres
10428167_D_lowres
10428142_D_lowres
10428146_D_lowres
10428160_D_lowres
10428167_D_lowres
10428142_D_lowres
10428178_D_lowres
33
10428175_D_lowres
33
10428141_D_lowres
33
10428146_D_lowres
10428141_D_lowres
10428174_D_lowres
33
10428169_D_lowres
10428159_D_lowres
33
10428172_D_lowres
10428177_D_lowres
33
10428158_D_lowres
10428175_D_lowres
10428168_D_lowres
10428137_D_lowres
33
10428142_D_lowres
10428158_D_lowres
10428141_D_lowres
10428142_D_lowres
10428151_D_lowres
10428170_D_lowres
33
10428175_D_lowres
10428151_D_lowres
10428167_D_lowres
10428172_D_lowres
10428151_D_lowres
10428139_D_lowres
10428168_D_lowres
10428151_D_lowres
10428143_D_lowres
10428147_D_lowres
33
10428146_D_lowres
10428151_D_lowres
10428145_D_lowres
10428174_D_lowres
10428157_D_lowres
10428141_D_lowres
10428175_D_lowres
10428147_D_lowre

34
10429016_D_lowres
34
10429012_D_lowres
34
10429016_D_lowres
10429016_D_lowres
10429012_D_lowres
10429024_D_lowres
34
10429012_D_lowres
10429021_D_lowres
34
10429024_D_lowres
10429013_D_lowres
34
10429021_D_lowres
10429016_D_lowres
10429021_D_lowres
10429012_D_lowres
10429021_D_lowres
10429012_D_lowres
10429016_D_lowres
10429021_D_lowres
10429024_D_lowres
10429026_D_lowres
34
10429012_D_lowres
10429024_D_lowres
10429021_D_lowres
10429015_D_lowres
10429012_D_lowres
10429016_D_lowres
10429015_D_lowres
10429013_D_lowres
10429026_D_lowres
10429024_D_lowres
10429026_D_lowres
10429013_D_lowres
10429026_D_lowres
10429021_D_lowres
10429015_D_lowres
10429021_D_lowres
10429012_D_lowres
10429021_D_lowres
10429012_D_lowres
10429012_D_lowres
10429012_D_lowres
10429024_D_lowres
10429015_D_lowres
10429016_D_lowres
10429013_D_lowres
10429024_D_lowres
10429013_D_lowres
10429021_D_lowres
10429015_D_lowres
10429015_D_lowres
10429026_D_lowres
10429016_D_lowres
10429015_D_lowres
10429024_D_lowres
1042902

9
10428096_D_lowres
9
10428109_D_lowres
10428094_D_lowres
9
10428089_D_lowres
9
10428097_D_lowres
9
10428101_D_lowres
9
10428089_D_lowres
10428089_D_lowres
10428095_D_lowres
9
10428094_D_lowres
10428089_D_lowres
10428089_D_lowres
10428095_D_lowres
10428096_D_lowres
10428098_D_lowres
9
10428094_D_lowres
10428096_D_lowres
10428096_D_lowres
10428098_D_lowres
10428098_D_lowres
10428093_D_lowres
9
10428109_D_lowres
10428109_D_lowres
10428093_D_lowres
10428096_D_lowres
10428098_D_lowres
10428098_D_lowres
10428097_D_lowres
10428095_D_lowres
10428097_D_lowres
10428109_D_lowres
10428089_D_lowres
10428095_D_lowres
10428109_D_lowres
10428095_D_lowres
10428109_D_lowres
10428098_D_lowres
10428109_D_lowres
10428089_D_lowres
10428095_D_lowres
10428097_D_lowres
10428098_D_lowres
10428089_D_lowres
10428089_D_lowres
melpomene_rosina_N_self
10427974_D_lowres
9
10427984_D_lowres
9
10427968_D_lowres
9
10427982_D_lowres
9
10427968_D_lowres
10427984_D_lowres
10427988_D_lowres
9
10427980_D_lowres
9
10427984_D

9
10428941_D_lowres
9
10428936_D_lowres
9
10428932_D_lowres
9
10428936_D_lowres
10428937_D_lowres
10428937_D_lowres
10428936_D_lowres
10428936_D_lowres
10428931_D_lowres
9
10428935_D_lowres
9
10428936_D_lowres
10428936_D_lowres
10428934_D_lowres
9
10428932_D_lowres
10428941_D_lowres
10428935_D_lowres
10428941_D_lowres
10428933_D_lowres
9
10428934_D_lowres
10428936_D_lowres
10428935_D_lowres
10428934_D_lowres
10428936_D_lowres
10428937_D_lowres
10428937_D_lowres
10428933_D_lowres
10428935_D_lowres
10428937_D_lowres
10428934_D_lowres
10428937_D_lowres
10428931_D_lowres
10428935_D_lowres
10428932_D_lowres
10428938_D_lowres
9
10428935_D_lowres
10428934_D_lowres
10428937_D_lowres
10428936_D_lowres
10428932_D_lowres
10428932_D_lowres
10428933_D_lowres
10428934_D_lowres
10428935_D_lowres
10428933_D_lowres
erato_favorinus_melpomene_amaryllis_cross
10429000_D_lowres
8
10429000_D_lowres
10429001_D_lowres
8
10428999_D_lowres
8
10428999_D_lowres
10428999_D_lowres
10429001_D_lowres
10429000_D_lowre

...

11
10428261_D_lowres
11
10428262_D_lowres
11
10428250_D_lowres
11
10428264_D_lowres
10428257_D_lowres
11
10428263_D_lowres
11
10428257_D_lowres
10428250_D_lowres
10428264_D_lowres
10428269_D_lowres
11
10428262_D_lowres
10428251_D_lowres
11
10428262_D_lowres
10428251_D_lowres
10428257_D_lowres
10428249_D_lowres
11
10428269_D_lowres
10428262_D_lowres
10428257_D_lowres
10428257_D_lowres
10428251_D_lowres
10428250_D_lowres
10428262_D_lowres
10428250_D_lowres
10428251_D_lowres
10428262_D_lowres
10428250_D_lowres
10428261_D_lowres
10428251_D_lowres
10428251_D_lowres
10428264_D_lowres
10428250_D_lowres
10428263_D_lowres
10428250_D_lowres
10428269_D_lowres
10428264_D_lowres
10428251_D_lowres
10428249_D_lowres
10428286_D_lowres
11
10428263_D_lowres
10428257_D_lowres
10428262_D_lowres
10428249_D_lowres
10428263_D_lowres
10428257_D_lowres
10428249_D_lowres
10428257_D_lowres
10428269_D_lowres
10428264_D_lowres
10428262_D_lowres
10428262_D_lowres
10428257_D_lowres
10428249_D_lowres
10428264_D_lowre

9
10428843_D_lowres
9
10428871_D_lowres
9
10428844_D_lowres
9
10428873_D_lowres
9
10428872_D_lowres
9
10428844_D_lowres
10428843_D_lowres
10428844_D_lowres
10428873_D_lowres
10428845_D_lowres
9
10428845_D_lowres
10428847_D_lowres
9
10428844_D_lowres
10428873_D_lowres
10428875_D_lowres
10428872_D_lowres
10428844_D_lowres
10428873_D_lowres
10428844_D_lowres
10428845_D_lowres
10428872_D_lowres
10428844_D_lowres
10428843_D_lowres
10428842_D_lowres
9
10428875_D_lowres
10428845_D_lowres
10428842_D_lowres
10428845_D_lowres
10428845_D_lowres
10428871_D_lowres
10428872_D_lowres
10428871_D_lowres
10428845_D_lowres
10428871_D_lowres
10428871_D_lowres
10428871_D_lowres
10428871_D_lowres
10428844_D_lowres
10428872_D_lowres
10428872_D_lowres
10428875_D_lowres
10428845_D_lowres
10428875_D_lowres
10428845_D_lowres
erato_petiverana_melpomene_rosina_S_cross
10428959_D_lowres
14
10428958_D_lowres
14
10428973_D_lowres
14
10428972_D_lowres
14
10428961_D_lowres
14
10428973_D_lowres
10428967_D_lowres
14
1042

2
10428484_D_lowres
2
10428484_D_lowres
erato_venus_melpomene_vulcanus_cross
10428844_D_lowres
11
10428849_D_lowres
11
10428842_D_lowres
11
10428849_D_lowres
10428871_D_lowres
11
10428873_D_lowres
11
10428845_D_lowres
11
10428872_D_lowres
11
10428872_D_lowres
10428873_D_lowres
10428875_D_lowres
11
10428844_D_lowres
10428847_D_lowres
11
10428871_D_lowres
10428872_D_lowres
10428872_D_lowres
10428842_D_lowres
10428842_D_lowres
10428844_D_lowres
10428842_D_lowres
10428842_D_lowres
10428875_D_lowres
10428842_D_lowres
10428872_D_lowres
10428845_D_lowres
10428842_D_lowres
10428844_D_lowres
10428871_D_lowres
10428849_D_lowres
10428873_D_lowres
10428844_D_lowres
10428845_D_lowres
10428871_D_lowres
10428872_D_lowres
10428842_D_lowres
10428842_D_lowres
10428845_D_lowres
10428849_D_lowres
10428847_D_lowres
10428847_D_lowres
10428844_D_lowres
10428843_D_lowres
11
10428849_D_lowres
10428844_D_lowres
10428843_D_lowres
10428849_D_lowres
10428843_D_lowres
10428843_D_lowres
10428843_D_lowres
10428875_D_

49
10428766_D_lowres
10428769_D_lowres
10428816_D_lowres
10428756_D_lowres
10428836_D_lowres
10428816_D_lowres
10428789_D_lowres
10428816_D_lowres
10428761_D_lowres
10428797_D_lowres
10428794_D_lowres
10428756_D_lowres
10428795_D_lowres
10428782_D_lowres
10428829_D_lowres
10428766_D_lowres
10428816_D_lowres
10428829_D_lowres
10428761_D_lowres
10428816_D_lowres
10428780_D_lowres
10428753_D_lowres
10428822_D_lowres
10428780_D_lowres
10428789_D_lowres
10428835_D_lowres
10428821_D_lowres
10428757_D_lowres
10428769_D_lowres
10428816_D_lowres
10428757_D_lowres
10428838_D_lowres
10428828_D_lowres
10428757_D_lowres
10428768_D_lowres
10428834_D_lowres
10428769_D_lowres
10428785_D_lowres
10428795_D_lowres
10428821_D_lowres
10428753_D_lowres
10428822_D_lowres
10428765_D_lowres
10428780_D_lowres
10428830_D_lowres
10428838_D_lowres
10428761_D_lowres
10428776_D_lowres
10428840_D_lowres
10428816_D_lowres
10428836_D_lowres
10428797_D_lowres
10428826_D_lowres
10428768_D_lowres
10428797_D_lowres
1042879

49
10428780_D_lowres
10428822_D_lowres
10428822_D_lowres
10428817_D_lowres
10428838_D_lowres
10428833_D_lowres
10428756_D_lowres
10428759_D_lowres
10428759_D_lowres
10428775_D_lowres
10428776_D_lowres
10428765_D_lowres
10428834_D_lowres
10428775_D_lowres
10428822_D_lowres
10428834_D_lowres
10428828_D_lowres
10428816_D_lowres
10428797_D_lowres
10428836_D_lowres
10428769_D_lowres
10428836_D_lowres
10428834_D_lowres
10428759_D_lowres
10428761_D_lowres
10428828_D_lowres
10428828_D_lowres
10428760_D_lowres
10428768_D_lowres
10428822_D_lowres
10428785_D_lowres
10428766_D_lowres
10428821_D_lowres
10428775_D_lowres
10428821_D_lowres
10428756_D_lowres
10428759_D_lowres
10428755_D_lowres
10428797_D_lowres
10428825_D_lowres
10428776_D_lowres
10428841_D_lowres
10428835_D_lowres
10428794_D_lowres
10428823_D_lowres
10428816_D_lowres
10428756_D_lowres
10428828_D_lowres
10428829_D_lowres
10428761_D_lowres
10428777_D_lowres
10428761_D_lowres
10428753_D_lowres
10428821_D_lowres
10428816_D_lowres
1042879

In [92]:
def getImage(file_name,folder_name,image_dir):
    print("Get into function: ",file_name,folder_name)
    for folder in os.scandir(image_dir+"/train/"):
        #print(folder.name , folder_name)
        if folder.name == folder_name:
            for file in os.scandir(folder.path):
                if file.name == file_name:
                    return file
    for folder in os.scandir(image_dir+"/test/"):
        if folder.name == folder_name:
            for file in os.scandir(folder):
                if file.name == file_name:
                    return file

In [ ]:
#plot heatmap
import matplotlib.pyplot as plt
from PIL import Image as im
from PIL import Image

notebook_path=os.path.dirname(os.path.realpath("Experiment4.ipynb"))
image_dir=notebook_path+"/ButterFly/"
npz_folder_location=notebook_path+"/Experiment4/SuperGlue_output_files/SuperGlue_output_files/"
heat_map_path=notebook_path+"/Experiment4/heat_map/"


with open ('mimic_pairs.csv') as csv_file:
    csv_reader=csv.reader(csv_file,delimiter=",")
    for row in csv_reader:
        #print(row[0],row[1])
        

        for folder in os.scandir(heat_map_path):
            if folder.name == row[0]+"_self":
                for file in os.scandir(folder.path):
                    
                    if file.name.endswith(".png"):
                        continue
                    
                    Image_path=getImage(file.name.rsplit( ".", 1 )[ 0 ]+".png",row[0],image_dir)
                    img=Image.open(Image_path.path)
                    wpercent = (1600/float(img.size[0]))
                    hsize = int((float(img.size[1])*float(wpercent)))
                    img = img.resize((1600,hsize), resample=Image.LANCZOS)
                
                    f, axarr = plt.subplots(2,1)
                    print(axarr.shape)
                    axarr[1].imshow(img)

                    npz=np.load(file)
                    heatmap=npz["heat_map"]
                    heatmap=np.divide(heatmap,np.sum(heatmap))

                    for i in range(1600):
                        for j in range(1600):
                            if heatmap[i][j]>0:
                                axarr[0].plot(j, i, marker='.', color="white")
                    axarr[0].imshow(img)
                    #plt.show()
                    plt.savefig(heat_map_path+folder.name+"/"+Image_path.name)
                    plt.close()
                    
                    
                    #search for it's corresponding cross
                    
                    for file_cross in os.scandir(heat_map_path+row[0]+"_"+row[1]+"_cross"):
                        if file_cross.name.endswith(".png"):
                            continue
                        if file_cross.name ==file.name:
                            f, axarr = plt.subplots(2,1)
                            print(axarr.shape)
                            axarr[1].imshow(img)

                            npz=np.load(file_cross)
                            cheatmap=npz["heat_map"]
                            difference_heatmap=heatmap
                            difference_heatmap[cheatmap>0]=0
                            difference_heatmap[difference_heatmap<0]=0

                            for i in range(1600):
                                for j in range(1600):
                                    if difference_heatmap[i][j]>0:
                                        axarr[0].plot(j, i, marker='.', color="red")
                            axarr[0].imshow(img)
                            plt.savefig(heat_map_path+row[0]+"_"+row[1]+"_cross/"+Image_path.name)
                            plt.close()
                            
                    
            elif folder.name == row[1]+"_self":
                for file in os.scandir(folder.path):
                    
                    if file.name.endswith(".png"):
                        continue
                    
                    Image_path=getImage(file.name.rsplit( ".", 1 )[ 0 ]+".png",row[1],image_dir)
                    img=Image.open(Image_path.path)
                    wpercent = (1600/float(img.size[0]))
                    hsize = int((float(img.size[1])*float(wpercent)))
                    img = img.resize((1600,hsize), resample=Image.LANCZOS)
                
                    f, axarr = plt.subplots(2,1)
                    print(axarr.shape)
                    axarr[1].imshow(img)

                    npz=np.load(file)
                    heatmap=npz["heat_map"]
                    heatmap=np.divide(heatmap,np.sum(heatmap))

                    for i in range(1600):
                        for j in range(1600):
                            if heatmap[i][j]>0:
                                axarr[0].plot(j, i, marker='.', color="white")
                    axarr[0].imshow(img)
                    #plt.show()
                    plt.savefig(heat_map_path+folder.name+"/"+Image_path.name)
                    plt.close()
                    
                    for file_cross in os.scandir(heat_map_path+row[0]+"_"+row[1]+"_cross"):
                        if file_cross.name.endswith(".png"):
                            continue
                        if file_cross.name ==file.name:
                            f, axarr = plt.subplots(2,1)
                            print(axarr.shape)
                            axarr[1].imshow(img)

                            npz=np.load(file_cross)
                            cheatmap=npz["heat_map"]
                            difference_heatmap=heatmap
                            difference_heatmap[cheatmap>0]=0
                            difference_heatmap[difference_heatmap<0]=0

                            for i in range(1600):
                                for j in range(1600):
                                    if difference_heatmap[i][j]>0:
                                        axarr[0].plot(j, i, marker='.', color="red")
                            axarr[0].imshow(img)
                            plt.savefig(heat_map_path+row[0]+"_"+row[1]+"_cross/"+Image_path.name)
                            plt.close()
                    #break



Get into function:  10427986_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427988_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427974_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427982_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427984_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427966_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427968_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427983_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10427980_D_lowres.png melpomene_rosina_N
(2,)
Get into function:  10429174_D_lowres.png erato_petiverana
(2,)
(2,)
Get into function:  10428959_D_lowres.png erato_petiverana
(2,)
(2,)
Get into function:  10429189_D_lowres.png erato_petiverana
(2,)
(2,)
Get into function:  10429167_D_lowres.png erato_petiverana
(2,)
(2,)
Get into function:  10429181_D_lowres.png erato_petiverana
(2,)
(2,)
Get into function:  10429182_D_lowres.png erato_petiverana
(2

(2,)
Get into function:  10428822_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428765_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428762_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428770_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428794_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428786_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428819_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428785_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428763_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428835_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428820_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428828_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428815_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428773_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  10428838_D_lowres.png erato_hydara
(2,)
(2,)
Get into function:  

Get into function:  10428236_D_lowres.png melpomene_ecuadorensis
(2,)
Get into function:  10428224_D_lowres.png melpomene_ecuadorensis
(2,)
Get into function:  10429104_D_lowres.png erato_emma
(2,)
(2,)
Get into function:  10428244_D_lowres.png melpomene_aglaope
(2,)
Get into function:  10428245_D_lowres.png melpomene_aglaope
(2,)
Get into function:  10428273_D_lowres.png melpomene_aglaope
(2,)
Get into function:  10428242_D_lowres.png melpomene_aglaope
(2,)
Get into function:  10428274_D_lowres.png melpomene_aglaope
(2,)
Get into function:  10428116_D_lowres.png melpomene_amaryllis
(2,)
Get into function:  10428119_D_lowres.png melpomene_amaryllis
(2,)
